In [12]:
!pip install tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 72.5 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml_dtypes 0.5.1
    Uninstalling ml_dtypes-0.5.1:
      Successfully uninstalled ml_dtypes-0.5.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jaxlib 0.6.2 requires ml_dtypes>=0.5.0, but you have ml-dtypes 0.4.1 which is incompatible.
jax 0.6.2 requires ml_dtypes>=0.5.0, but you have ml-dtypes 0.4.1 which is incompatible.


In [13]:
!pip install opencv-python

In [14]:

!pip uninstall -y mediapipe numpy
!pip cache purge
!pip install numpy mediapipe --no-cache-dir --force-reinstall



Found existing installation: mediapipe 0.10.21
Uninstalling mediapipe-0.10.21:
  Successfully uninstalled mediapipe-0.10.21
Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
Files removed: 6
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 78.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 184.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 151.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.9/107.9 kB 173.4 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of opencv-contrib-python to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 199.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 310.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 264.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:

import os
import time
import numpy as np
import scipy.stats
import random
import pandas as pd


import matplotlib.pyplot as plt
from IPython.display import display, Image

import cv2
import mediapipe as mp
from google.colab.patches import cv2_imshow

from sklearn.model_selection import train_test_split
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.models import load_model

/usr/local/lib/python3.11/dist-packages/jaxlib/plugin_support.py:71: RuntimeWarning: JAX plugin jax_cuda12_plugin version 0.5.1 is installed, but it is not compatible with the installed jaxlib version 0.6.2, so it will not be used.
  warnings.warn(


In [15]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [16]:

base_dir = "/content/drive/MyDrive/sign_language_detection"

test_videos_path = os.path.join(base_dir, 'test_videos')
if not os.path.exists(test_videos_path):
    os.makedirs(test_videos_path)

DATA_PATH = os.path.join(base_dir, 'MP_Data')

In [4]:

mp_holistic = mp.solutions.holistic


mp_drawing = mp.solutions.drawing_utils


def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results


def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

def draw_styled_landmarks(image, results):

    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             )

    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             )

    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             )

In [9]:

video_name = "IMAGE_VEDIO1.mp4"
video_path = os.path.join(test_videos_path, video_name)


cap = cv2.VideoCapture(video_path)


if not cap.isOpened():
    raise FileNotFoundError(f"Error: Cannot open video file '{video_path}'. Please check if the file exists and the path is correct.")
mp_holistic = mp.solutions.holistic
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)

        if not ret:
            break


        image, results = mediapipe_detection(frame, holistic)
        draw_styled_landmarks(image, results)


        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)


        plt.figure(figsize=(10, 10))
        plt.imshow(image_rgb)
        plt.axis('off')
        display(plt.gcf())
        plt.close()

    # cv2.waitKey(1)
    cap.release()

Output hidden; open in https://colab.research.google.com to view.

In [10]:

pose = []


for res in results.pose_landmarks.landmark:

    test = np.array([res.x, res.y, res.z, res.visibility])


    pose.append(test)


pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)


face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)

def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [11]:
actions = np.array(['hello', 'thank_you', 'see_you_later'])

DATA_PATH = os.path.join(base_dir, 'MP_Data')
VIDEO_PATH = os.path.join(base_dir, 'videos')

if not os.path.exists(DATA_PATH):
    os.makedirs(DATA_PATH)

no_sequences = 20
sequence_length = 20
start_folder = 20

for action in actions:

    action_path = os.path.join(DATA_PATH, action)
    video_path = os.path.join(VIDEO_PATH, action)


    if not os.path.exists(action_path):
        os.makedirs(action_path)


    if not os.path.exists(video_path):
        os.makedirs(video_path)


    existing_dirs = [d for d in os.listdir(action_path) if d.isdigit()]
    if existing_dirs:
        dirmax = np.max(np.array(existing_dirs).astype(int))
    else:
        dirmax = 0


    for sequence in range(0, no_sequences):
        seq_path = os.path.join(action_path, str(dirmax + sequence))
        if not os.path.exists(seq_path):
            os.makedirs(seq_path)

In [ ]:
!git config --global user.email "draashish2003malik@gmail.com"
!git config --global user.name "Aashish"


In [12]:

base_video_path = os.path.join(base_dir, 'videos')

if not os.path.exists(base_video_path):
    os.makedirs(base_video_path)

for action in actions:

    action_video_path = os.path.join(base_video_path, action)
    action_data_path = os.path.join(DATA_PATH, action)
    os.makedirs(action_data_path, exist_ok=True)


    for i in range(no_sequences):
        sequence_folder = os.path.join(action_data_path, str(i))
        os.makedirs(sequence_folder, exist_ok=True)

    video_counter = 0
    for video_file in os.listdir(action_video_path):
        print(f"\n▶ Processing {video_file}")

        full_video_path = os.path.join(action_video_path, video_file)


        cap = cv2.VideoCapture(full_video_path)
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        print(f"Total frames: {total_frames}")

        target_frame_count = 20
        #20
        frame_indices = np.linspace(0, total_frames - 1, target_frame_count, dtype=int)

        with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
            for i, frame_index in enumerate(frame_indices):
                cap.set(cv2.CAP_PROP_POS_FRAMES, frame_index)
                ret, frame = cap.read()
                if not ret:
                    print(f"⚠ Could not read frame {frame_index}")
                    continue


                image, results = mediapipe_detection(frame, holistic)
                draw_styled_landmarks(image, results)


                keypoints = extract_keypoints(results)


                npy_path = os.path.join(action_data_path, str(i), f"{video_counter}.npy")
                np.save(npy_path, keypoints)
                print(f"✅ Saved to {npy_path}")


        cap.release()
        video_counter += 1


▶ Processing WIN_20250720_22_21_03_Pro.mp4
Total frames: 34
✅ Saved to /content/drive/MyDrive/sign_language_detection/MP_Data/hello/0/0.npy
✅ Saved to /content/drive/MyDrive/sign_language_detection/MP_Data/hello/1/0.npy
✅ Saved to /content/drive/MyDrive/sign_language_detection/MP_Data/hello/2/0.npy
✅ Saved to /content/drive/MyDrive/sign_language_detection/MP_Data/hello/3/0.npy
✅ Saved to /content/drive/MyDrive/sign_language_detection/MP_Data/hello/4/0.npy
✅ Saved to /content/drive/MyDrive/sign_language_detection/MP_Data/hello/5/0.npy
✅ Saved to /content/drive/MyDrive/sign_language_detection/MP_Data/hello/6/0.npy
✅ Saved to /content/drive/MyDrive/sign_language_detection/MP_Data/hello/7/0.npy
✅ Saved to /content/drive/MyDrive/sign_language_detection/MP_Data/hello/8/0.npy
✅ Saved to /content/drive/MyDrive/sign_language_detection/MP_Data/hello/9/0.npy
✅ Saved to /content/drive/MyDrive/sign_language_detection/MP_Data/hello/10/0.npy
✅ Saved to /content/drive/MyDrive/sign_language_detection/

In [13]:
label_map = {label: num for num, label in enumerate(actions)}

label_map

{'hello': 0, 'thank_you': 1, 'see_you_later': 2}

In [17]:
sequences, labels = [], []


for action in actions:

    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []


        for frame_num in range(sequence_length):

            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)


        sequences.append(window)


        labels.append(label_map[action])

X = np.array(sequences)
y = to_categorical(labels).astype(int)

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)


print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

X_train shape: (85, 20, 1662)
X_test shape: (5, 20, 1662)
y_train shape: (85, 3)
y_test shape: (5, 3)


In [19]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

model = Sequential()

model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))

model.add(LSTM(128, return_sequences=True, activation='relu'))

model.add(LSTM(64, return_sequences=False, activation='relu'))

model.add(Dense(64, activation='relu'))

model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [20]:
model.compile(optimizer='Adam',

              loss='categorical_crossentropy',

              metrics=['categorical_accuracy'])

In [21]:
model.fit(X_train, y_train, epochs=200, callbacks=[tb_callback])

Epoch 1/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 8s 150ms/step - categorical_accuracy: 0.3366 - loss: 1.1818
Epoch 2/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 148ms/step - categorical_accuracy: 0.3522 - loss: 1.6033
Epoch 3/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step - categorical_accuracy: 0.4109 - loss: 1.1414
Epoch 4/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 84ms/step - categorical_accuracy: 0.3405 - loss: 1.7196
Epoch 5/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - categorical_accuracy: 0.3072 - loss: 1.3002
Epoch 6/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step - categorical_accuracy: 0.2818 - loss: 1.1752
Epoch 7/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step - categorical_accuracy: 0.1684 - loss: 1.1123
Epoch 8/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - categorical_accuracy: 0.4443 - loss: 1.1022
Epoch 9/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step - categorical_accuracy: 0.3386 - loss: 1.0940
Epoch 10/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step - categorical_accuracy: 0.3601 - loss: 1.0572
Epoch 11/200
3/3 ━━━━━━━━━━━━━━━━━━━━

In [22]:
y_pred = model.predict(X_test)

y_pred.shape

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 456ms/step


(5, 3)

In [23]:
actions[np.argmax(y_pred[3])]

'thank_you'

In [24]:
actions[np.argmax(y_test[3])]

'thank_you'

In [25]:
model_path = os.path.join(base_dir, 'my_model.keras')
model.save(model_path)

In [26]:
model_path = os.path.join(base_dir, 'my_model.keras')
model = load_model(model_path)

In [27]:
y_pred = model.predict(X_test)
y_true = np.argmax(y_test, axis=1).tolist()
y_pred = np.argmax(y_pred, axis=1).tolist()

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 658ms/step


In [28]:
conf_matrix = multilabel_confusion_matrix(y_true, y_pred)

for idx, matrix in enumerate(conf_matrix):
    print(f"\nConfusion Matrix for class '{actions[idx]}':")
    df = pd.DataFrame(matrix,
                      index=["Actual Negative", "Actual Positive"],
                      columns=["Predicted Negative", "Predicted Positive"])
    print(df)


Confusion Matrix for class 'hello':
                 Predicted Negative  Predicted Positive
Actual Negative                   2                   0
Actual Positive                   0                   3

Confusion Matrix for class 'thank_you':
                 Predicted Negative  Predicted Positive
Actual Negative                   3                   0
Actual Positive                   0                   2


In [29]:
accuracy_score(y_true, y_pred)

1.0

In [34]:

test_video = "IMAGE_VEDIO4.mp4" # TODO: Change to the name of your video
base_test_path = "/content/drive/MyDrive/sign_language_detection/test_videos"
input_video_path = os.path.join(base_test_path, test_video)
output_video_path = os.path.join(base_test_path, "output_video5.mp4")
os.makedirs(base_test_path, exist_ok=True)

colors = [tuple(random.randint(0, 255) for _ in range(3)) for _ in actions]

def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()

    for num, prob in enumerate(res):
        color = colors[num]
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), color, -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)

    return output_frame



sequence = []
sentence = []
predictions = []
threshold = 0.5


cap = cv2.VideoCapture(input_video_path)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))


mp_holistic = mp.solutions.holistic
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break


        frame = cv2.flip(frame, 1)


        image, results = mediapipe_detection(frame, holistic)
        draw_styled_landmarks(image, results)


        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]

        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            predictions.append(np.argmax(res))

            if np.unique(predictions[-10:])[0] == np.argmax(res):
                if res[np.argmax(res)] > threshold:
                    if len(sentence) == 0 or actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5:
                sentence = sentence[-5:]

            image = prob_viz(res, actions, image, colors)

        cv2.rectangle(image, (0, 0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)


        out.write(image)

    cap.release()
    out.release()
    # cv2.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━